In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

In [2]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [12]:
def generate_model(p):
  input_layer = layers.Input(shape=(200, 4))

  conv1 = layers.Conv1D(filters=64, kernel_size=13, padding='same', use_bias='false')(input_layer)
  BN1 = layers.BatchNormalization()(conv1)
  relu1 = layers.Activation('relu')(BN1)
  pool1 = layers.MaxPool1D(pool_size=p)(relu1)
  drop1 = layers.Dropout(0.05)(pool1)

  positions = tf.range(drop1.shape[1])
  context = layers.Embedding(input_dim=drop1.shape[1], output_dim=drop1.shape[2])(positions)

  contextual_meaning = tf.add(drop1, context)

  query = layers.Dense(100, name='query')(contextual_meaning)
  value = layers.Dense(100)(contextual_meaning)

  attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32, dropout=0.55)(query, value, return_attention_scores=True)

  flat = layers.Flatten()(attention)
  fcl = layers.Dense(512, activation='relu')(flat)
  BN2 = layers.BatchNormalization()(fcl)
  relu2 = layers.Activation('relu')(BN2)
  drop2 = layers.Dropout(0.5)(fcl)

  output = layers.Dense(12, activation='sigmoid')(drop2)

  model = Model(inputs=input_layer, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
  return model

In [11]:
model = generate_model(10)
model.fit(x_train, y_train, epochs=60, validation_data=(x_valid, y_valid))

Epoch 1/60
657/657 [==============================] - 7s 9ms/step - loss: 0.5113 - auroc: 0.5175 - aupr: 0.1512 - val_loss: 0.5085 - val_auroc: 0.5761 - val_aupr: 0.2050
Epoch 2/60
657/657 [==============================] - 5s 8ms/step - loss: 0.4201 - auroc: 0.6091 - aupr: 0.2381 - val_loss: 0.3887 - val_auroc: 0.6816 - val_aupr: 0.3254
Epoch 3/60
657/657 [==============================] - 5s 8ms/step - loss: 0.3842 - auroc: 0.6905 - aupr: 0.3532 - val_loss: 0.3207 - val_auroc: 0.8053 - val_aupr: 0.5280
Epoch 4/60
657/657 [==============================] - 5s 8ms/step - loss: 0.3356 - auroc: 0.7806 - aupr: 0.5019 - val_loss: 0.2721 - val_auroc: 0.8637 - val_aupr: 0.6402
Epoch 5/60
657/657 [==============================] - 5s 7ms/step - loss: 0.3045 - auroc: 0.8256 - aupr: 0.5805 - val_loss: 0.2561 - val_auroc: 0.8819 - val_aupr: 0.6803
Epoch 6/60
657/657 [==============================] - 5s 8ms/step - loss: 0.2862 - auroc: 0.8522 - aupr: 0.6293 - val_loss: 0.2306 - val_auroc: 0.9095

KeyboardInterrupt: ignored